In [2]:
import gradio as gr
from llama_index import VectorStore, ResponseSynthesizer
from llama_index.llms.ollama import Ollama
from llama_index.embeddings.huggingface import HuggingFaceEmbedding

# Initialize components
def initialize_system():
    # Configure embedding model
    embedding_model = HuggingFaceEmbedding(model_name="BAAI/bge-small-en-v1.5")
    
    # Initialize Vector Store
    vector_index = VectorStore(embedding_model=embedding_model)
    
    # Configure LLM (Phi3.5 via Ollama)
    llm = Ollama(model_name="phi3.5")
    
    # Combine into a synthesizer
    query_engine = ResponseSynthesizer(llm=llm, vector_index=vector_index)
    return vector_index, query_engine

vector_index, query_engine = initialize_system()

# File upload handler
def process_file(file):
    if file is not None:
        try:
            # Read and index file
            reader = SimpleDirectoryReader(input_dir=file.name)
            documents = reader.load_data()
            vector_index.add_documents(documents)
            return f"Successfully processed and indexed: {file.name}"
        except Exception as e:
            return f"Error processing file: {str(e)}"
    return "No file uploaded."

# Query handler
def query_document(query, history):
    try:
        response = query_engine.query(query)
        history.append((query, response.response))
        return history, response.response
    except Exception as e:
        return history, f"Error: {str(e)}"

# Gradio Interface
with gr.Blocks() as demo:
    gr.Markdown("# 🧠 Document Search and Conversation System")
    
    with gr.Tab("Upload & Index"):
        file_input = gr.File(label="Upload Document", type="file")
        upload_output = gr.Textbox(label="Output")
        upload_button = gr.Button("Process File")
        upload_button.click(process_file, inputs=file_input, outputs=upload_output)
    
    with gr.Tab("Query"):
        query_input = gr.Textbox(label="Your Query", placeholder="Ask something...")
        conversation_output = gr.Chatbot(label="Conversation History")
        query_button = gr.Button("Search")
        query_button.click(query_document, inputs=[query_input, conversation_output], outputs=[conversation_output, "response"])
        
    with gr.Tab("About"):
        gr.Markdown("This interface allows you to upload documents and perform semantic search.")

# Launch Gradio App
demo.launch()


AttributeError: 'Examples' object has no attribute 'render'